In [1]:
import os
import re
import glob
import zipfile
import fnmatch
import datetime
from zipfile import ZipFile
import pandas as pd
import numpy as np
from pandas import DataFrame
import openpyxl
from openpyxl import load_workbook

In [2]:
def create_timestamp():
    """
    Creates a timestamp in DB format.
    """
    today = datetime.date.today()
    year = today.year
    month = today.month
    day = today.day
    
    timestamp = f"{str(year)}-{str(month)}-{str(day)}"
    
    return timestamp

In [3]:
data_dir = ".\\excel\\data\\"
archive_dir = ".\\excel\\archive\\"

if not os.path.exists(archive_dir):
    os.makedirs(archive_dir)
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

In [19]:
xl_list = glob.glob(data_dir + "*.xlsx")

for xl_file in xl_list:
    sheets = pd.ExcelFile(xl_file).sheet_names
    workbook = pd.ExcelFile(xl_file)

    if fnmatch.fnmatch(xl_file.lower(), "*reason*.xlsx") == True:
        df_reason = workbook.parse(0, skiprows=2, header=None)
        df_reason.columns = ["dept", 
                             "category", 
                             "itemDesc", 
                             "itemCode", 
                             "outOfStock", 
                             "manufacIssue",
                             "disc",
                             "other",
                             "newItemIssue"]
        df_reason["itemCode"] = df_reason["itemCode"].map('{:0>6}'.format)
        df_reason["max"] = df_reason[[df_reason.columns[4], 
                                      df_reason.columns[5], 
                                      df_reason.columns[6], 
                                      df_reason.columns[7], 
                                      df_reason.columns[8]]].max(axis=1)
        df_reason.loc[df_reason["max"] == df_reason["outOfStock"], "primaryReason"] = "Out Of Stock"
        df_reason.loc[df_reason["max"] == df_reason["manufacIssue"], "primaryReason"] = "Manufacturer Issue"
        df_reason.loc[df_reason["max"] == df_reason["disc"], "primaryReason"] = "Discontinued"
        df_reason.loc[df_reason["max"] == df_reason["other"], "primaryReason"] = "Other"
        df_reason.loc[df_reason["max"] == df_reason["newItemIssue"], "primaryReason"] = "New Item Issue"
        df_reason.sort_values(by=["max"], ascending=False, inplace=True)
        df_reason.drop(columns=["dept", 
                                "category", 
                                "itemDesc", 
                                "outOfStock", 
                                "manufacIssue", 
                                "disc", 
                                "other", 
                                "newItemIssue", 
                                "max"], inplace=True)
        
    elif fnmatch.fnmatch(xl_file.lower(), "*short*.xlsx") == True:
        df_shorts = workbook.parse(0, skiprows=1, header=None)
        df_shorts.columns = ["itemDesc", 
                             "itemCode", 
                             "yesterdayOOS"]
        df_shorts["itemCode"] = df_shorts["itemCode"].map('{:0>6}'.format)
        df_shorts.drop(columns=["itemDesc"], inplace=True)
        
    elif fnmatch.fnmatch(xl_file.lower(), "*base*.xlsx") == True:
        df_base = workbook.parse(0, skiprows=1, header=None)
        df_base.columns = ["dept", 
                           "category", 
                           "itemDesc", 
                           "itemCode", 
                           "itemSize", 
                           "pvtLblFlag", 
                           "buyerCode", 
                           "invUnitShipped", 
                           "invCaseShipped", 
                           "storeOrdProdQty", 
                           "shortedQty", 
                           "grossSvcLvl", 
                           "netSvcLvl"]
        df_base["itemCode"] = df_base["itemCode"].map('{:0>6}'.format)
        df_base["buyerCode"] = df_base["buyerCode"] * 10
        df_base["itemDesc"] = df_base["itemDesc"] + "   " + df_base["itemSize"]
        
    elif fnmatch.fnmatch(xl_file.lower(), "*export*.xlsx") == True:
        to_drop = ["14:HATFIELD NORTH", "1:BRATTLEBORO"]
        
        df_cs = workbook.parse(0, skiprows=3, skipfooter=20, header=None)
        df_cs = df_cs[~df_cs[7].isin(to_drop)]
        df_cs = df_cs.filter([0, 14, 15, 17, 34, 7])
        df_cs.columns = ["custCode", 
                         "poDueDate", 
                         "poApptDate", 
                         "inStock", 
                         "daysOOS", 
                         "dest"]
        df_cs["itemCode"] = df_cs["custCode"].astype(str).str[9:15]
        df_cs.drop(columns=["custCode"], inplace=True)
        df_cs.drop_duplicates(inplace=True)

In [13]:
# to_drop = ["14:HATFIELD NORTH", "1:BRATTLEBORO"]
# df_cs = df_cs[~df_cs[7].isin(to_drop)]
df_cs

,poDueDate,poApptDate,inStock,daysOOS,dest,itemCode
0,2020-07-27,2020-07-27,N,4.0,46:WNY: GDC + PDC,459493
2,2020-07-21,2020-07-23,N,6.0,46:WNY: GDC + PDC,281836
5,2020-07-29,2020-07-29,N,5.0,47:WNY: FREEZER,041961
9,NaT,NaT,N,31.0,46:WNY: GDC + PDC,270147
10,NaT,NaT,N,11.0,47:WNY: FREEZER,135018
12,2020-07-27,NaT,N,2.0,8:MAULDIN,037720
14,NaT,NaT,N,6.0,46:WNY: GDC + PDC,204425
15,2020-07-27,NaT,N,5.0,8:MAULDIN,343576
17,NaT,NaT,N,10.0,47:WNY: FREEZER,130055
19,NaT,NaT,N,32.0,46:WNY: GDC + PDC,272653


In [18]:
df_join_1 = df_base.merge(df_reason, how="left", on="itemCode")
df_join_2 = df_join_1.merge(df_shorts, how="left", on="itemCode")
df_join_3 = df_join_2.merge(df_cs, how="left", on="itemCode")

# df_join_2["poDueDate"] = "NO CS DATA"
# df_join_2["poApptDate"] = "NO CS DATA"
# df_join_2["inStock"] = "NO CS DATA"
# df_join_2["daysOOS"] = "NO CS DATA"
# df_join_2
df_join_3['poDueDate'].fillna('NO CS DATA')
df_join_3

,dept,category,itemDesc,itemCode,itemSize,pvtLblFlag,buyerCode,invUnitShipped,invCaseShipped,storeOrdProdQty,shortedQty,grossSvcLvl,netSvcLvl,primaryReason,yesterdayOOS,poDueDate,poApptDate,inStock,daysOOS,dest
0,HOUSEHOLD,PAPER TOWELS,BOUNTY TOWEL SAS 6B=8REG RL 199 SF,278875,199 SF,N,210,-98,-98,1259,1357,-0.077840,1.000000,Manufacturer Issue,1361,2020-07-29,2020-07-29,N,2.0,46:WNY: GDC + PDC
1,HOUSEHOLD,PAPER TOWELS,SMPLY DN PAPER TWL SS WHT 6RL 305.02 SF,270140,305.02 SF,Y,210,37,37,617,580,0.059968,0.880952,Manufacturer Issue,584,2020-07-29,2020-07-29,N,1.0,46:WNY: GDC + PDC
2,HOUSEHOLD,PAPER TOWELS,SMPLY DN ULT WHT TWL6H=15R I/O 373.10 SF,270189,373.10 SF,Y,210,-7,-7,464,471,-0.015086,1.000000,Manufacturer Issue,476,2020-07-29,NaT,N,6.0,46:WNY: GDC + PDC
3,BEVERAGE SODA,CARBONATED BEVERAGES,TOPS 12PK SODA ORANGE 144 FZ,281193,144 FZ,Y,160,0,0,452,452,0.000000,NaN,Manufacturer Issue,454,2020-07-21,2020-07-23,N,11.0,46:WNY: GDC + PDC
4,HOUSEHOLD,PAPER TOWELS,SMPLY DN ULT SAS TWL8G=20R I/O 497.50 SF,270188,497.50 SF,Y,210,-19,-19,425,444,-0.044706,1.000000,Manufacturer Issue,444,2020-07-29,NaT,N,7.0,46:WNY: GDC + PDC
5,BEVERAGE SODA,CARBONATED BEVERAGES,TOPS 12PK SODA ROOT BEER 144 FZ,281836,144 FZ,Y,160,-1,-1,409,410,-0.002445,-0.002451,Out Of Stock,413,2020-07-21,2020-07-23,N,6.0,46:WNY: GDC + PDC
6,BEVERAGE SODA,CARBONATED BEVERAGES,TOPS 12PK SODA GINGERALE 144 FZ,281844,144 FZ,Y,160,-2,-2,390,392,-0.005128,1.000000,Manufacturer Issue,392,2020-07-21,2020-07-23,N,6.0,46:WNY: GDC + PDC
7,HOUSEHOLD,BATH TISSUE,SMPLY DN BATH TIS 1000CT 20RL 2106.94 SF,270138,2106.94 SF,Y,210,0,0,371,371,0.000000,NaN,Manufacturer Issue,376,NaT,NaT,N,17.0,46:WNY: GDC + PDC
8,HOUSEHOLD,PAPER TOWELS,BOUNTY SAS WHITE 8DBL+ =20RL 496 SF,274888,496 SF,N,210,-1,-1,369,370,-0.002710,1.000000,Manufacturer Issue,370,2020-07-28,2020-07-28,N,4.0,46:WNY: GDC + PDC
9,MEAT FROZEN PROCESSED,BACON,J DEAN APPLEWOOD SMKD BACON 16 OZ,459493,16 OZ,N,510,0,0,354,354,0.000000,0.000000,Out Of Stock,354,2020-07-27,2020-07-27,N,4.0,46:WNY: GDC + PDC


In [ ]:
df_join_3.to_excel(f".\\excel\\archive\\oos-data-{create_timestamp()}.xlsx", index=False)

In [ ]:
# template = r".\\excel\\template.xlsx"

# writer = pd.ExcelWriter(template, engine='openpyxl')
# df_join_2.to_excel(writer, "Data", index=False)
# writer.save()
# writer.close()

# book = load_workbook(template)
# writer = pd.ExcelWriter(template, engine='openpyxl')
# writer.book = book
# df_join_2.to_excel(writer, sheet_name = "Data", index=False)
# writer.save()
# writer.close()

# workbook1 = openpyxl.load_workbook(".\\excel\\template.xlsx")

# writer = pd.ExcelWriter('file.xlsx', engine='openpyxl') 
# writer.book = workbook1
# df_join_2.to_excel(writer, sheet_name='Data', index=False, startrow=2, startcol=1)
 
# writer.save()
# writer.close()

# with pd.ExcelWriter(".\\excel\\template.xlsx", engine='openpyxl', mode='a') as writer:  
#     df_join_2.to_excel(writer, sheet_name='Data')
#     writer.close()